# Getting Funky: MONDian Gravity

One of the awesome things about <tt>cluster_generator</tt> is that you can use non-Newtonian gravities. We've implemented several MOND based theories for you to use and explore, but other can be constructed with relative ease. In this guide, we're going to walk you through all of the things you need to know about using non-Newtonian dynamics.

---

## Contents

- [Important Notes for Users](#Important-Note)
- [Getting Started With Gravity](#Getting-Started-With-Gravity)

---

## Important Note

<tt>cluster_generator</tt> is written in a modular style to facilitate further development both by the development team or by the user. We've made it really easy to implement new gravity theories; **however**, it should be noted that very few simulation softwares actually support non-Newtonian gravity options. In this guide, we will be setting up initial conditions for use in RAMSES because their is a patch, called [RaYMOND](https://github.com/graemecan/raymond_qumond), which implements a MOND solver in RAMSES. For other, more arcane gravity theories, the user may need to write a simulation package for themselves (hard work!) or utilize the other available workflows that are available from <tt>cluster_generator</tt> (simulated X-ray observations, analytical work, etc.) to probe the science problem of interest.

## Getting Started With Gravity

Now that we've talked about the limitations, we can start getting into the fun stuff. All of the gravitational theories available in the <tt>cluster_generator</tt> library are found in the [gravity](../../_as_gen/gravity.html) module. The <tt>gravity</tt> module contains classes representing each of the available gravitational theories. For each of the gravitational theories, two methods are defined, one which computes the potential, and another which computes the dynamical mass from hydrostatic equilibrium.

For the most part, you really shouldn't need to interact directly with any of these modules. Instead, you can specify your preferred gravitational theory using the ``gravity=`` keyword when generating ``ClusterModel``'s. Let's see this in action! We'll start by just building a reference model in Newtonian gravity.
<br>
<br>


In [1]:
# -- Imports -- #
import cluster_generator as cg
from cluster_generator.utils import cgparams

cgparams["system"]["text"]["spinners"] = False # --> Turn off spinners so they look okay in documentation.

# Redshift for cluster
z = 0.546


# masses for primary cluster
M500 = 1.1e15
M200 = M500 * 1.23

# This helps us find r200 for the two clusters
r200_1 = cg.find_overdensity_radius(M200, 200.0, z=z)

# Compute parameters for truncated NFW profile
r_s1 = r200_1 / 2
rho_s1 = cg.nfw_scale_density(2, z=z)
r_t1 = r200_1 * 2.0


# This creates truncated NFW mass profile objects
# these are like functions, you can call them, e.g. Mt_1(r)
Mt_1 = cg.tnfw_mass_profile(rho_s1, r_s1, r_t1)

# This finds true radii and masses based on the profile
r500_1, M500_1 = cg.find_radius_mass(Mt_1, z=z, delta=500.0)
r2500_1, M2500_1 = cg.find_radius_mass(Mt_1, z=z, delta=2500.0)

rhot_1 = cg.tnfw_density_profile(rho_s1, r_s1, r_t1)

f_g = 0.115
rhog_1 = cg.vikhlinin_density_profile(1.0, 0.5 * r2500_1, 1.1 * r200_1, 0.1, 0.67, 3)
rhog_1 = cg.rescale_profile_by_mass(rhog_1, f_g * M500_1, r500_1)

model_Newtonian = cg.HydrostaticEquilibrium.from_dens_and_tden(0.1, 15000.0, rhog_1, rhot_1)



cluster_generator : [INFO     ] 2023-09-20 07:49:34,398 Computing the profiles from density and total density. Gravity=Newtonian
cluster_generator : [INFO     ] 2023-09-20 07:49:34,403 Integrating total mass profile.
cluster_generator : [INFO     ] 2023-09-20 07:49:34,703 ClusterModel [ClusterModel object; gravity=Newtonian] has no virialization method. Setting to default = eddington
cluster_generator : [INFO     ] 2023-09-20 07:49:34,704 Computing gravitational potential of ClusterModel object; gravity=Newtonian. gravity=Newtonian.
cluster_generator : [INFO     ] 2023-09-20 07:49:35,843 Integrating pressure profile.


TypeError: _from_scratch() got multiple values for keyword argument 'stellar_density'

Now, we can do the exact same thing (with a little less code) to make our MOND cluster. The only thing that get's changed is the ``gravity="QUMOND"`` keyword argument when we generate the model:

In [ ]:
model_mond = cg.HydrostaticEquilibrium.from_dens_and_tden(0.1,15000.0,rhog_1,rhot_1,gravity="QUMOND")

<div class="alert alert-block alert-info" style="background-color: white; border: 2px solid; padding: 10px">
<p><b> &#10068; What in the Log!?</b></p> 

If you look closely, there are some clear differences in the logging output from these two generation methods. The most glaring are the warnings about the "interp_function" and "a_0".

<u>These should be expected</u> and are only there to alert the user that they didn't specify these gravity theory specific parameters directly. If you want to specify one or both of these parameters, specify it directly in the kwargs of the model generating method.
</div>

You notice that the output from the log has changed somewhat, but things went more or less the same. We can start comparing our two clusters now by generating plots of different fields:

In [ ]:
import matplotlib.pyplot as plt

figure,axes = plt.subplots(2,2,figsize=(10,8),gridspec_kw={"wspace":0.7,"hspace":0.4})

fields = ["temperature","density","dark_matter_density","total_density"]
names = [r"Temperature",r"$\rho_g$",r"$\rho_{dm}$",r"$\rho_{dyn}$"]

for f,a,n in zip(fields,axes.ravel(),names):
    model_mond.plot(f,fig=figure,ax=a,color="darkblue")
    model_Newtonian.plot(f,fig=figure,ax=a,color="forestgreen")
    a.set_xlabel("radius, [kpc]")
    a.set_ylabel(f"{n}, {model_mond[f].units}")
axes[0,0].set_yscale("linear")


plt.show()

As you can see, the $\rho_{dm}$, $\rho_{dyn}$ and $\rho_{gas}$ are the same in both systems (because we initialized the clusters from the same profiles), but the temperatures are dramatically different. This is due to the gravitational differences.

<div class="alert alert-block alert-info" style="background-color: white; border: 2px solid; padding: 10px">
<p><b> &#10068; How Does it Work?</b></p> 

Like all of the models in <tt>cluster_generator</tt>, these profiles are determined by enforcing hydrostatic equilibrium in the clusters. In MOND, the gravitational acceleration in the outer regions of the cluster is <u>larger than in the Newtonian equivalent</u> and therefore increases the necessary pressure to maintain HSE and therefore also increases the temperature.
</div>
<br>
<br>
Now, we should probably check that both of our models are without any physicality issues:

In [ ]:
q = model_Newtonian.is_physical()
print(q[0])
q = model_mond.is_physical()
print(q[0])


Awesome! Both of these clusters are physically realizable!

<div class="alert alert-block alert-warning" style="background-color: white; border: 2px solid; padding: 10px">
<p><b> &#128679; Important Note:</b></p> 

Unlike in Newtonian gravity, many of the profiles we provide are <u>not specifically designed for MOND.</u> This means that cluster models generated in MOND gravity will often have non-physical behaviors at large radii. 

<u>Always check your clusters</u>
</div>

## Making Initial Conditions

---

Now that we've built our cluster, there's very little left to show. You can procede as usual to generate the initial conditions and the rest. Here, we'll create some particle initial conditions so that we can look at the differences in the two clusters. First, we'll have to save our clusters to file.

In [ ]:
model_mond.write_model_to_h5("mond_model.h5",overwrite=True)
model_Newtonian.write_model_to_h5("newtonian_model.h5",overwrite=True)

Now we can start building the initial conditions:


In [ ]:
mond_ics = cg.ClusterICs("mond_ics",1,["mond_model.h5"],[[0,0,0]],[[0,0,0]],num_particles={"dm":1_000_000},r_max=5000)
newt_ics = cg.ClusterICs("newt_ics",1,["newtonian_model.h5"],[[0,0,0]],[[0,0,0]],num_particles={"dm":1_000_000},r_max=5000)

mond_parts = mond_ics.create_dataset([256,256,256],10000,left_edge=[-5000,-5000,-5000])
newt_parts = newt_ics.create_dataset([256,256,256],10000,left_edge=[-5000,-5000,-5000])

In [ ]:
import yt
p = yt.SlicePlot(mond_parts,"z",("gas","kT"),width=(5000,"kpc"))
p.set_log(("gas","kT"),False)
p.set_cmap(("gas",'kT'),"hot")
p.show()
p = yt.SlicePlot(newt_parts,"z",("gas","kT"),width=(5000,"kpc"))
p.set_log(("gas","kT"),False)
p.set_cmap(("gas",'kT'),"hot")
p.show()